In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
%cd /notebooks
datadir = "data/"
kaggledir = datadir + "kaggle_catch-me-if-you-can"

import numpy as np
import pandas as pd
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
"""
import os
for dirname, _, filenames in os.walk(datadir):
    for filename in filenames:
        print(os.path.join(dirname, filename))
"""
# Any results you write to the current directory are saved as output.

/notebooks


'\nimport os\nfor dirname, _, filenames in os.walk(datadir):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n'

In [2]:
# a helper function for writing predictions to a file
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [3]:
train_df = pd.read_csv(kaggledir + '/train_sessions.csv',
                       index_col='session_id')
test_df = pd.read_csv(kaggledir + '/test_sessions.csv',
                      index_col='session_id')

# Convert time1, ..., time10 columns to datetime type
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# Sort the data by time
train_df = train_df.sort_values(by='time1')

# Look at the first rows of the training set
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


In [4]:
train_df.shape, test_df.shape

((253561, 21), (82797, 20))

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 21669 to 204762
Data columns (total 21 columns):
site1     253561 non-null int64
time1     253561 non-null datetime64[ns]
site2     250098 non-null float64
time2     250098 non-null datetime64[ns]
site3     246919 non-null float64
time3     246919 non-null datetime64[ns]
site4     244321 non-null float64
time4     244321 non-null datetime64[ns]
site5     241829 non-null float64
time5     241829 non-null datetime64[ns]
site6     239495 non-null float64
time6     239495 non-null datetime64[ns]
site7     237297 non-null float64
time7     237297 non-null datetime64[ns]
site8     235224 non-null float64
time8     235224 non-null datetime64[ns]
site9     233084 non-null float64
time9     233084 non-null datetime64[ns]
site10    231052 non-null float64
time10    231052 non-null datetime64[ns]
target    253561 non-null int64
dtypes: datetime64[ns](10), float64(9), int64(2)
memory usage: 42.6 MB


In [6]:
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites].fillna(0).astype('int').to_csv(kaggledir + '/train_sessions_text.txt', 
                                               sep=' ', 
                       index=None, header=None)
test_df[sites].fillna(0).astype('int').to_csv(kaggledir + '/test_sessions_text.txt', 
                                              sep=' ', 
                       index=None, header=None)

In [7]:
!head -5 data/kaggle_catch-me-if-you-can/train_sessions_text.txt

56 55 0 0 0 0 0 0 0 0
56 55 56 55 0 0 0 0 0 0
946 946 951 946 946 945 948 784 949 946
945 948 949 948 945 946 947 945 946 946
947 950 948 947 950 952 946 951 946 947


In [8]:
!ls -ls data/kaggle_catch-me-if-you-can/

total 99876
  780 -rw-r--r-- 1 501 80   797372 Jun  9 23:32 32132_205932_compressed_site_dic.pkl.zip
 2804 -rw-r--r-- 1 501 80  2298301 Jun  9 22:38 logit_sub001.txt
 2636 -rw-r--r-- 1 501 80  2308088 Jun  9 23:14 logit_sub002.txt
 1204 -rw-rw-r-- 1 501 80  1230867 Nov 28  2019 sample_submission.csv
18996 -rw-rw-r-- 1 501 80 19450729 Oct 17  2019 test_sessions.csv
 4100 -rw-r--r-- 1 501 80  3313920 Jun 17 17:07 test_sessions_text.txt
59112 -rw-rw-r-- 1 501 80 60526801 Oct 17  2019 train_sessions.csv
10244 -rw-r--r-- 1 501 80 10098117 Jun 17 17:07 train_sessions_text.txt


In [9]:
cv = CountVectorizer()

#### digression for bag of words

In [19]:
cv.fit_transform(["site1 site17 site2",
                  "site2 site2 site1"]).todense

<bound method spmatrix.todense of <2x3 sparse matrix of type '<class 'numpy.int64'>'
	with 5 stored elements in Compressed Sparse Row format>>

In [24]:
X_sparse = cv.fit_transform(["this movie is awful, is this movie",
                             "enjoyed this movie"])

In [25]:
X_sparse.nonzero(), X_sparse.data

((array([0, 0, 0, 0, 1, 1, 1], dtype=int32),
  array([4, 3, 2, 0, 4, 3, 1], dtype=int32)),
 array([2, 2, 2, 1, 1, 1, 1]))

#### end digression; read material in Topic 4. Linear models. Part 4. Pros & Cons

In [10]:
%%time
with open(kaggledir + '/train_sessions_text.txt') as inp_train_file:
    X_train = cv.fit_transform(inp_train_file)
with open(kaggledir + '/test_sessions_text.txt') as inp_test_file:
    X_test = cv.transform(inp_test_file)
print(X_train.shape, X_test.shape)

(253561, 41592) (82797, 41592)
CPU times: user 28.8 s, sys: 2.03 s, total: 30.8 s
Wall time: 31.7 s


In [11]:
type(X_train)

scipy.sparse.csr.csr_matrix

In [12]:
y_train = train_df['target'].astype(int)

In [13]:
y_train.head()

session_id
21669     0
54843     0
77292     0
114021    0
146670    0
Name: target, dtype: int64

### train Logistic regression

In [14]:
logit = LogisticRegression(C = 1, random_state=17)

In [15]:
%%time
cv_scores = cross_val_score(logit, X_train, y_train, cv=5, scoring='roc_auc')

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

CPU times: user 2min 14s, sys: 13.1 s, total: 2min 27s
Wall time: 1min 18s


In [16]:
cv_scores

array([0.91381466, 0.82974406, 0.87640038, 0.89222744, 0.91358078])

In [17]:
cv_scores.mean()

0.8851534657470413

In [18]:
%%time
logit.fit(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


CPU times: user 38.6 s, sys: 3.92 s, total: 42.5 s
Wall time: 22.7 s


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=17, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [19]:
test_pred_logit1 = logit.predict_proba(X_test)[:,1]

In [20]:
test_pred_logit1

array([2.42986531e-03, 5.10600347e-09, 1.88419235e-08, ...,
       8.84323390e-03, 4.74556322e-04, 2.26304950e-05])

In [37]:
write_to_submission_file(test_pred_logit1, kaggledir + '/logit_sub001.txt') ## .908 ROC AUC; Kaggle score 0.90804

In [21]:
!head data/kaggle_catch-me-if-you-can/logit_sub001.txt

session_id,target
1,0.002429865306031476
2,5.106003468943695e-09
3,1.884192350275818e-08
4,3.001812552507454e-08
5,3.344002873506393e-05
6,0.0002914539011159398
7,0.0008029991308606806
8,0.00016645636782992783
9,0.0009129090870587053


### Time Features

- hour when the session started
- morning 
- day
- eve
- night

In [22]:
def add_time_features(time1_series, X_sparse):
    hour = time1_series.apply(lambda ts: ts.hour)
    morning = ((hour >= 7) & (hour <= 11)).astype('int')
    day = ((hour >= 12) & (hour <= 18)).astype('int')
    evening = ((hour >= 19) & (hour <= 23)).astype('int')
    night = ((hour >= 0) & (hour <= 6)).astype('int')
    # stack features together
    X = hstack([X_sparse, morning.values.reshape(-1, 1), # why reshape?
                day.values.reshape(-1, 1),
                evening.values.reshape(-1, 1), 
                night.values.reshape(-1, 1)])
    return X

In [23]:
test_df.loc[:, 'time1'].fillna(0).apply(lambda ts: ts.hour).head()

session_id
1    11
2    11
3    15
4    10
5    15
Name: time1, dtype: int64

In [24]:
%%time
X_train_with_time = add_time_features(train_df['time1'].fillna(0), X_train)

CPU times: user 4.57 s, sys: 123 ms, total: 4.69 s
Wall time: 4.76 s


In [25]:
%%time
X_test_with_time = add_time_features(test_df['time1'].fillna(0), X_test)

CPU times: user 1.07 s, sys: 6.85 ms, total: 1.08 s
Wall time: 1.11 s


In [26]:
X_train_with_time.shape, X_test_with_time.shape

((253561, 41596), (82797, 41596))

In [27]:
logit_with_time = LogisticRegression(C = 1, random_state=17)

In [28]:
%%time
cv_scores = cross_val_score(logit_with_time, X_train_with_time, y_train, cv= 5, scoring='roc_auc');

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

CPU times: user 3min 40s, sys: 26 s, total: 4min 6s
Wall time: 2min 16s


In [29]:
cv_scores

array([0.92516138, 0.90530537, 0.93167379, 0.94366538, 0.94800725])

In [30]:
cv_scores.mean()

0.9307626346605634

In [31]:
%%time
logit_with_time.fit(X_train_with_time, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


CPU times: user 24.6 s, sys: 2.3 s, total: 26.9 s
Wall time: 14.4 s


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=17, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [32]:
test_pred_logit2 = logit_with_time.predict_proba(X_test_with_time)[:,1]

In [33]:
test_pred_logit2

array([7.08401052e-05, 6.15457538e-08, 3.87858434e-08, ...,
       3.11409184e-04, 1.51321141e-05, 6.85065623e-07])

In [56]:
write_to_submission_file(test_pred_logit2, kaggledir + '/logit_sub002.txt') ## .93565 ROC AUC; Kaggle score 0.93565

In [34]:
!head data/kaggle_catch-me-if-you-can/logit_sub002.txt

session_id,target
1,7.084010524001853e-05
2,6.154575381443169e-08
3,3.878584335628161e-08
4,1.2572817198632427e-09
5,4.524283118795582e-05
6,8.767070335029314e-06
7,0.001744445772799691
8,0.00036413504933000115
9,3.78799089970287e-05


### 2590 (of 4123) Winston Lee
### You advanced 1,351 places on the leaderboard!
### Your submission scored 0.93565, which is an improvement of your previous score of 0.90804.

## watch end of video

In [35]:
c_values = np.logspace(-2, 2, 10)
c_values

array([1.00000000e-02, 2.78255940e-02, 7.74263683e-02, 2.15443469e-01,
       5.99484250e-01, 1.66810054e+00, 4.64158883e+00, 1.29154967e+01,
       3.59381366e+01, 1.00000000e+02])

In [38]:
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

time_split = TimeSeriesSplit(n_splits=10)

logit_grid = GridSearchCV(estimator=logit, param_grid={'C': c_values},
                          scoring="roc_auc", cv=time_split, n_jobs=-1, verbose=True)


In [40]:
logit_grid.fit(X_train_with_time, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 19.0min finished


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=10),
             error_score='raise-deprecating',
             estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=17, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'C': array([1.00000000e-02, 2.78255940e-02, 7.74263683e-02, 2.15443469e-01,
       5.99484250e-01, 1.66810054e+00, 4.64158883e+00, 1.29154967e+01,
       3.59381366e+01, 1.00000000e+02])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False

In [41]:
logit_grid.best_score_, logit_grid.best_params_

(0.9155313015135583, {'C': 0.5994842503189409})

In [42]:
test_pred_logit3 = logit_grid.predict_proba(X_test_with_time)[:,1]

In [43]:
write_to_submission_file(test_pred_logit3, kaggledir + '/logit_sub003.txt') ## 0.9155 ROC AUC; Kaggle score 0.93740

### 2553 (of 4123) Winston Lee
### You advanced 43 places on the leaderboard!
### roc_auc = 0.9155313015135583; Kaggle = 0.93740
### Your submission scored 0.93740, which is an improvement of your previous score of 0.93565.